In [ ]:
base_network=resnet.ResNet50(weights="imagenet",input_shape=(200,200,3),include_top=False)
for layer in base_network.layers[:-3]:
  layer.trainable=False
pool=GlobalAveragePooling2D()(base_network.output)
dense1=Dense(128,activation='tanh')(pool)
output=Dense(128)(dense1)
embedding=Model(base_network.input,output,name="Embedding")

In [ ]:
def euclidean_distance(vects):
  x,y=vects
  sum_square=K.sum(K.square(x-y),axis=1,keepdims=True)
  return K.sqrt((sum_square))

In [ ]:
img_1=Input(shape=(200,200,3),name="input_1")
img_2=Input(shape=(200,200,3),name="input_2")

feat_1=embedding(img_1)
feat_2=embedding(img_2)

distance=Lambda(euclidean_distance)([feat_1,feat_2])

model=Model(inputs=[img_1,img_2],outputs=distance)


In [ ]:
def contrastive_loss(y_true,y_pred,margin=1):
  squared_distance_similar=K.square(y_pred)
  squared_distance_nonsimilar=k.square(K.maximum(margin-y_pred,0))
  loss=K.mean(y_true*squared_distance_similar+(1-y_true)*(squared_distance_nonsimilar))

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Flatten, Dense
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import os

# Load your dataframe (example)
df = pd.read_csv('image_pairs.csv')  # Assumes 'image_1' and 'image_2' columns with paths

# Image dimensions (same for both images)
IMAGE_SIZE = (224, 224)  # Modify as per your model's input size

# Helper function to load and preprocess images
def preprocess_image(img_path, target_size=IMAGE_SIZE):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)  # If using ResNet
    return img_array

# Function to create the Siamese Network
def build_siamese_model(input_shape=(224, 224, 3)):
    # Base model using ResNet50 (you can use any model)
    base_model = ResNet50(include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    # Adding custom layers on top of base model
    flatten = Flatten()(base_model.output)
    dense = Dense(128, activation="relu")(flatten)
    output = Dense(64, activation="sigmoid")(dense)

    # Create the model using the base model and the custom layers
    model = Model(inputs=base_model.input, outputs=output)
    return model

# Lambda layer for computing Euclidean distance
def euclidean_distance(vectors):
    (feat1, feat2) = vectors
    sum_square = tf.reduce_sum(tf.square(feat1 - feat2), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, 1e-6))

# Contrastive loss function (if using it for training)
def contrastive_loss(y_true, y_pred, margin=1):
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

# Data generator that yields image pairs and labels
def data_generator(df, batch_size=32):
    while True:
        for start in range(0, len(df), batch_size):
            batch = df[start:start + batch_size]
            img1 = np.array([preprocess_image(img) for img in batch['image_1']])
            img2 = np.array([preprocess_image(img) for img in batch['image_2']])
            labels = np.array(batch['label'])  # Assuming 'label' column in the DataFrame
            yield [img1, img2], labels

# Build the model
input_shape = (224, 224, 3)
siamese_network = build_siamese_model(input_shape)

# Create the model with the distance layer
input_1 = Input(input_shape)
input_2 = Input(input_shape)

# Create the shared model
shared_model = siamese_network

# Get the embeddings
output_1 = shared_model(input_1)
output_2 = shared_model(input_2)

# Calculate the Euclidean distance
distance = Lambda(euclidean_distance)([output_1, output_2])

# Define the full Siamese model
siamese_model = Model(inputs=[input_1, input_2], outputs=distance)

# Compile the model with contrastive loss and an optimizer
siamese_model.compile(loss=contrastive_loss, optimizer=Adam(learning_rate=0.0001))

# Assuming you have a CSV like this:
# image_1          | image_2          | label
# /path/to/img1    | /path/to/img2    | 0
# /path/to/img3    | /path/to/img4    | 1
# where 'label' = 0 means the images are not similar, 'label' = 1 means they are similar

# Train the model using the data generator
siamese_model.fit(data_generator(df), epochs=10, steps_per_epoch=len(df) // 32)
